In [84]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 50)
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import precision_recall_cutoff
from cost_function import cost_function, cost_function_cutoff
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox
from sklearn.svm import SVC, SVR
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, classification_report, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV


# Reading the csv file
train = pd.read_csv('train_dataset.csv')

# Defining the file to be read from s3 bucket
test = pd.read_csv('test_dataset.csv')
test = test.dropna()
test.shape

(498121, 44)

In [85]:
train.shape

(1879, 45)

In [86]:
train.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud,interaction_1,interaction_2,interaction_3,interaction_4,average_seconds_per_item,trustLevel_1,trustLevel_2,trustLevel_3,trustLevel_4,trustLevel_5,trustLevel_6,totalScanTimeInSeconds_0_1,grandTotal_0_1,lineItemVoids_0_1,scansWithoutRegistration_0_1,quantityModifications_0_1,scannedLineItemsPerSecond_0_1,valuePerSecond_0_1,lineItemVoidsPerPosition_0_1,average_seconds_per_item_01,Labels,Labels_0,Labels_1,Labels_2,Labels_3,Labels_4,Labels_5,interaction_5,interaction_6,interaction_7,interaction_8,heredity_1,heredity_2,heredity_3,interaction_9
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0,0,0,0,1,19.27,0,0,0,0,1,0,0.575178,0.547174,0.636364,0.0,0.6,0.004045,0.001370,0.021944,0.000131,0,1,0,0,0,0,0,0.000000,0.002427,0.314722,0,0,0,0,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0,0,0,0,1,3.95,0,0,1,0,0,0,0.057955,0.273637,0.454545,0.2,0.8,0.019364,0.006689,0.032468,0.000027,1,0,1,0,0,0,0,0.090909,0.015491,0.015859,0,0,0,3,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0,0,0,0,1,24.39,0,0,1,0,0,0,0.827775,0.621811,0.272727,1.0,1.0,0.001204,0.001083,0.020979,0.000165,5,0,0,0,0,0,1,0.272727,0.001204,0.514719,0,0,0,15,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0,0,0,0,1,19.40,0,0,0,0,0,1,0.978130,0.923462,0.727273,0.4,0.8,0.002347,0.001361,0.025078,0.000132,2,0,0,1,0,0,0,0.290909,0.001877,0.903266,0,0,0,12,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0,0,0,0,1,5.27,0,0,0,0,1,0,0.234008,0.815608,0.272727,0.7,0.4,0.009337,0.005007,0.010101,0.000036,4,0,0,0,0,1,0,0.190909,0.003735,0.190858,0,0,0,20,0


In [87]:
# Defining input and target variables
X = train[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9', 'heredity_2', 'interaction_5']]
Y = train['fraud']

# Splitting the data into train, test, and validation
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2)
X_val, X_test, Y_val, Y_test = train_test_split(X_val, Y_val, test_size = 0.2)


# Scaling the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns = X.columns)

X_test = scaler.fit_transform(X_test)
X_test = pd.DataFrame(X_test, columns = X.columns)

X_val = scaler.fit_transform(X_val)
X_val = pd.DataFrame(X_val, columns = X.columns)


X_val

,totalScanTimeInSeconds,interaction_1,interaction_4,heredity_1,interaction_9,heredity_2,interaction_5
0,0.614537,0.0,0.0,0.0,0.0,0.0,0.127273
1,0.079295,0.0,1.0,0.0,0.0,0.0,0.300000
2,1.000000,0.0,0.0,0.0,0.0,0.0,0.290909
3,0.787996,0.0,1.0,0.0,0.0,0.0,0.318182
4,0.569934,0.0,0.0,0.0,0.0,0.0,0.136364
...,...,...,...,...,...,...,...
295,0.824339,0.0,0.0,0.0,0.0,0.0,0.018182
296,0.812225,0.0,1.0,0.0,0.0,0.0,0.327273
297,0.063326,0.0,1.0,0.0,0.0,0.0,0.181818
298,0.704295,0.0,1.0,0.0,0.0,0.0,0.500000


In [88]:
# Defining top 7, 6, and 5 variables
# Train dataset
X_train_7 = X_train
X_train_6 = X_train[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9', 'heredity_2']]
X_train_5 = X_train[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9']]

# Test dataset
X_test_7 = X_test
X_test_6 = X_test[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9', 'heredity_2']]
X_test_5 = X_test[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9']]

# Validation dataset
X_val_7 = X_val
X_val_6 = X_val[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9', 'heredity_2']]
X_val_5 = X_val[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9']]


### Random Forest Model

### Fitting the model with the top 5 features

In [47]:
## Defining the hyper-parameters for RF
RF_param_grid = {'n_estimators': [100, 300, 500],
                 'min_samples_split': [10, 15], 
                 'min_samples_leaf': [5, 7], 
                 'max_depth' : [3, 5, 7]}

# Defining customized scoring function
my_score_function = make_scorer(cost_function, greater_is_better = True, needs_proba = True)

# Performing GridSearch
RF_grid_search = GridSearchCV(RandomForestClassifier(), RF_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train_5, Y_train)

# Extracting the best model
RF_model = RF_grid_search.best_estimator_
RF_model

RandomForestClassifier(max_depth=3, min_samples_leaf=7, min_samples_split=10,
                       n_estimators=500)

In [48]:
RF_grid_search.cv_results_

{'mean_fit_time': array([0.35863527, 0.86803571, 1.10768716, 0.21665716, 0.64600531,
        1.1555291 , 0.23778892, 0.65637136, 1.07568598, 0.21680331,
        0.64102912, 1.12585537, 0.23267738, 0.70796363, 1.08664044,
        0.22108308, 0.6766475 , 1.13175774, 0.23073657, 0.74226809,
        1.08864148, 0.22042958, 0.66091053, 1.07916554, 0.22709672,
        0.78834422, 1.10755777, 0.22579352, 0.65764165, 1.09369477,
        0.22392869, 0.70219072, 1.13958526, 0.22379692, 0.65865405,
        0.92788585]),
 'std_fit_time': array([0.06195   , 0.04749   , 0.07055066, 0.00086038, 0.00595501,
        0.06300926, 0.01973585, 0.006165  , 0.00600896, 0.00247938,
        0.00633941, 0.03661542, 0.01822118, 0.03888567, 0.01698715,
        0.00195799, 0.00560626, 0.01789966, 0.00956147, 0.03412162,
        0.01087494, 0.00404717, 0.00612233, 0.00763909, 0.00695814,
        0.00898224, 0.03628873, 0.0078083 , 0.00817015, 0.00914258,
        0.00362574, 0.03823365, 0.08325399, 0.00381633, 0.005

### Score

In [49]:
score = RF_grid_search.cv_results_
score['mean_test_score'][0]

-75.0

### Fitting the model with the top 6 features

In [50]:
# Defining customized scoring function
my_score_function = make_scorer(cost_function, greater_is_better = True, needs_proba = True)

# Performing GridSearch
RF_grid_search = GridSearchCV(RandomForestClassifier(), RF_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train_6, Y_train)

# Extracting the best model
RF_model = RF_grid_search.best_estimator_
RF_model

RandomForestClassifier(max_depth=3, min_samples_leaf=7, min_samples_split=15,
                       n_estimators=300)

### Score

In [51]:
score = RF_grid_search.cv_results_
score['mean_test_score'][0]

-75.0

### Fitting the model with the top 7 features

In [52]:
# Defining customized scoring function
my_score_function = make_scorer(cost_function, greater_is_better = True, needs_proba = True)

# Performing GridSearch
RF_grid_search = GridSearchCV(RandomForestClassifier(), RF_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train_7, Y_train)

# Extracting the best model
RF_model = RF_grid_search.best_estimator_
RF_model

RandomForestClassifier(max_depth=3, min_samples_leaf=5, min_samples_split=15,
                       n_estimators=500)

### Score

In [53]:
score = RF_grid_search.cv_results_
score['mean_test_score'][0]

-78.33333333333333

## From above three scenarios, the best model is the following:
#### RandomForestClassifier(max_depth=3, min_samples_leaf=7, min_samples_split=10, n_estimators=500) with the top 5 features.

### Support Vector Machine

### Fitting the model with the top 5 features

In [59]:
## Defining the hyper-parameters for svc
svc_param_grid = {'kernel': ['rbf', 'poly', 'sigmoid'], 
                  'C': [0.01, 0.1, 1, 10],
                  'gamma': [0.001, 0.01, 0.1, 1]}

# Defining customized scoring function
my_score_function = make_scorer(cost_function, greater_is_better = True, needs_proba = True)

svc_grid_search = GridSearchCV(SVC(probability = True), svc_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train_5, Y_train)

# Extracting the best model
svc_md = svc_grid_search.best_estimator_
svc_md

SVC(C=0.01, gamma=0.1, kernel='sigmoid', probability=True)

### Score

In [60]:
score = svc_grid_search.cv_results_
score['mean_test_score'][0]

-63.333333333333336

### Fitting the model with the top 6 features

In [61]:
svc_grid_search = GridSearchCV(SVC(probability = True), svc_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train_6, Y_train)

# Extracting the best model
svc_md = svc_grid_search.best_estimator_
svc_md

SVC(C=0.1, gamma=1, kernel='poly', probability=True)

### Score

In [62]:
score = svc_grid_search.cv_results_
score['mean_test_score'][0]

-83.33333333333333

### Fitting the model with the top 7 features

In [63]:
svc_grid_search = GridSearchCV(SVC(probability = True), svc_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train_7, Y_train)

# Extracting the best model
svc_md = svc_grid_search.best_estimator_
svc_md

SVC(C=0.01, gamma=1, probability=True)

### Score

In [64]:
score = svc_grid_search.cv_results_
score['mean_test_score'][0]

-65.0

## From above three scenarios, the best model is the following:
#### SVC(C=0.01, gamma=0.1, kernel='sigmoid', probability=True) with the top 5 features.

### AdaBoost

### Fitting the model with the top 5 features

In [65]:
## Defining the hyper-parameters for RF
Ada_param_grid = {'n_estimators': [100, 300, 500],
                 'base_estimator__min_samples_split': [10, 15], 
                 'base_estimator__min_samples_leaf': [5, 7], 
                 'base_estimator__max_depth' : [3, 5, 7],
                 'learning_rate': [0.001, 0.01, 0.1]}

## Running grid search with 3 fold
Ada_grid_search = GridSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), Ada_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train_5, Y_train)

# Extracting the best model
ada_md = Ada_grid_search.best_estimator_
ada_md

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3,
                                                         min_samples_leaf=5,
                                                         min_samples_split=10),
                   learning_rate=0.01, n_estimators=300)

### Score

In [66]:
score = Ada_grid_search.cv_results_
score['mean_test_score'][0]

-121.66666666666667

### Fitting the model with the top 6 features

In [67]:
## Running grid search with 3 fold
Ada_grid_search = GridSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), Ada_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train_6, Y_train)

# Extracting the best model
ada_md = Ada_grid_search.best_estimator_
ada_md

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3,
                                                         min_samples_leaf=5,
                                                         min_samples_split=15),
                   learning_rate=0.01, n_estimators=300)

### Score

In [68]:
score = Ada_grid_search.cv_results_
score['mean_test_score'][0]

-118.33333333333333

### Fitting the model with the top 7 features

In [69]:
## Running grid search with 3 fold
Ada_grid_search = GridSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), Ada_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train_7, Y_train)

# Extracting the best model
ada_md = Ada_grid_search.best_estimator_
ada_md

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3,
                                                         min_samples_leaf=5,
                                                         min_samples_split=10),
                   learning_rate=0.01, n_estimators=300)

### Score

In [71]:
score = Ada_grid_search.cv_results_
score['mean_test_score'][0]

-106.66666666666667

## From above three scenarios, the best model is the following:
#### AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3, min_samples_leaf=5, min_samples_split=10), learning_rate=0.01, n_estimators=300) with the top 7 features.

## From above models, the best model so far is the following:
#### SVC(C=0.01, gamma=0.1, kernel='sigmoid', probability=True) with the top 5 features.

### Fitting the model with the top 5 features

In [89]:
from xgboost import XGBClassifier
## Defining the hyper-parameters for RF
XGBoost_param_grid = {'n_estimators': [500],
                        'max_depth': [3, 5, 7],
                        'min_child_weight': [5, 7],
                        'learning_rate': [0.01],
                        'gamma': [0.3, 0.1],
                        'subsample': [0.8, 1],
                        'colsample_bytree': [1]}

# Defining customized scoring function
my_score_function = make_scorer(cost_function, greater_is_better = True, needs_proba = True)

# Performing GridSearch
XGBoost_grid_search = GridSearchCV(XGBClassifier(use_label_encoder = False, eval_metric = 'logloss'), XGBoost_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train_5, Y_train)

# Extracting the best model
XGBoost_model = XGBoost_grid_search.best_estimator_
XGBoost_model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='logloss', gamma=0.3, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.01, max_delta_step=0,
              max_depth=5, min_child_weight=5, missing=nan,
              monotone_constraints='()', n_estimators=500, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

### Score

In [90]:
score = XGBoost_grid_search.cv_results_
score['mean_test_score'][0]

-80.0

### Fitting the model with the top 6 features

In [91]:
# Defining customized scoring function
my_score_function = make_scorer(cost_function, greater_is_better = True, needs_proba = True)

# Performing GridSearch
XGBoost_grid_search = GridSearchCV(XGBClassifier(use_label_encoder = False, eval_metric = 'logloss'), XGBoost_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train_6, Y_train)

# Extracting the best model
XGBoost_model = XGBoost_grid_search.best_estimator_
XGBoost_model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='logloss', gamma=0.3, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.01, max_delta_step=0,
              max_depth=3, min_child_weight=7, missing=nan,
              monotone_constraints='()', n_estimators=500, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

### Score

In [92]:
score = XGBoost_grid_search.cv_results_
score['mean_test_score'][0]

-75.0

### Fitting the model with the top 7 features

In [ ]:
# Defining customized scoring function
my_score_function = make_scorer(cost_function, greater_is_better = True, needs_proba = True)

# Performing GridSearch
XGBoost_grid_search = GridSearchCV(XGBClassifier(use_label_encoder = False, eval_metric = 'logloss'), XGBoost_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train_7, Y_train)

# Extracting the best model
XGBoost_model = XGBoost_grid_search.best_estimator_
XGBoost_model

### Score

In [ ]:
score = XGBoost_grid_search.cv_results_
score['mean_test_score'][0]